In [ ]:
import cv2 
import mediapipe as mp 
import time 
import requests
import numpy as np
import math 

mpHands =mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

#chnage this url according yours 
url = "http://192.168.1.2:8080/shot.jpg"
overlayList = []

img1 = cv2.imread('finger1.png')
overlayList.append(img1)
img2 = cv2.imread('finger2.png')
overlayList.append(img2)
img3 = cv2.imread('finger3.png')
overlayList.append(img3)
img4 = cv2.imread('finger4.png')
overlayList.append(img4)
img5 = cv2.imread('finger5.png')
overlayList.append(img5)
img6 = cv2.imread('finger6.png')
overlayList.append(img6)


for i in range(len(overlayList)):
    overlayList[i] = cv2.resize(overlayList[i],(200,200))



pTime = 0
cTime = 0

lmList = []
tipIds = [4, 8, 12, 16, 20]

while True:
    # Getting Raw data
    RawData = requests.get(url, verify=False)

    # Convertting it to serilized one deminsion array
    One_D_Arry = np.array(bytearray(RawData.content),dtype = np.uint8)
    
    # converting One deminsion Array into opencv image matrxi, format using "imdecode" function 
    frame = cv2.imdecode(One_D_Arry, -1)
    frame = cv2.resize(frame,(640,480))
    imgRGB = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB)
    #print(results.multi_hand_landmarks)
    
    if results.multi_hand_landmarks:
        
        for handlm in results.multi_hand_landmarks:
            for id , lm in enumerate(handlm.landmark):
                #print(id,lm)
                cx,cy = int(lm.x*512),int(lm.y*512)
                lmList.append([id,cx,cy])
                #print(id,cx,cy)
                if id==0:
                    cv2.circle(frame,(cx,cy),10,(255,30,30),cv2.FILLED)
            mpDraw.draw_landmarks(frame,handlm,mpHands.HAND_CONNECTIONS)
    
    
    cTime =  time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime 
    cv2.putText(frame,str(int(fps)),(10,40),cv2.FONT_HERSHEY_COMPLEX,3,(164,123,44),3)
    # show image on the screen, using imshow function of opencv
    
    if(len(lmList)!=0):

        fingers = []

        # Thumb
        if lmList[tipIds[0]][1] > lmList[tipIds[0] - 1][1]:
            fingers.append(1)
        else:
            fingers.append(0)

        # 4 Fingers
        for id in range(1, 5):
            if lmList[tipIds[id]][2] > lmList[tipIds[id] - 2][2]:
                fingers.append(1)
            else:
                fingers.append(0)

        print(fingers)
        totalFingers = fingers.count(0)
        print(totalFingers)

        h, w, c = overlayList[totalFingers - 1].shape
        frame[0:h, 0:w] = overlayList[totalFingers - 1]

        cv2.rectangle(frame, (20, 225), (170, 425), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, str(totalFingers), (45, 375), cv2.FONT_HERSHEY_PLAIN,
                    10, (255, 0, 0), 25)

    lmList = []  
    cv2.imshow("window", frame)
    # break out of loop when/if "q" key press on the keyborad by using waitKey fuction of opencv
    if cv2.waitKey(1)==ord('q'):
        break

cv2.destroyAllWindows()
    
    